In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zopc99r7
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zopc99r7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=2d8b7b5534eb4a1b4961c50eb6dafbf277569d08cb4c5b74d8f7de19bbc0054e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_c_oplj3/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


#**HELLO WORLD in CUDA**

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define NUM_BLOCKS 16
#define BLOCK_WIDTH 2

__global__ void hello(){
    printf("Hello World from GPU's \t blockDim:%d \t blockNo: %d \t threadNo: %d \t threadId: %d\n",blockDim.x,blockIdx.x,threadIdx.x,(blockDim.x*blockIdx.x+threadIdx.x));
}

int main() {
    printf("Hello World from CPU \n");
    hello<<<NUM_BLOCKS,BLOCK_WIDTH>>>();
    cudaDeviceSynchronize(); 
    return 0;
}

Hello World from CPU 
Hello World from GPU's 	 blockDim:2 	 blockNo: 12 	 threadNo: 0 	 threadId: 24
Hello World from GPU's 	 blockDim:2 	 blockNo: 12 	 threadNo: 1 	 threadId: 25
Hello World from GPU's 	 blockDim:2 	 blockNo: 15 	 threadNo: 0 	 threadId: 30
Hello World from GPU's 	 blockDim:2 	 blockNo: 15 	 threadNo: 1 	 threadId: 31
Hello World from GPU's 	 blockDim:2 	 blockNo: 7 	 threadNo: 0 	 threadId: 14
Hello World from GPU's 	 blockDim:2 	 blockNo: 7 	 threadNo: 1 	 threadId: 15
Hello World from GPU's 	 blockDim:2 	 blockNo: 14 	 threadNo: 0 	 threadId: 28
Hello World from GPU's 	 blockDim:2 	 blockNo: 14 	 threadNo: 1 	 threadId: 29
Hello World from GPU's 	 blockDim:2 	 blockNo: 10 	 threadNo: 0 	 threadId: 20
Hello World from GPU's 	 blockDim:2 	 blockNo: 10 	 threadNo: 1 	 threadId: 21
Hello World from GPU's 	 blockDim:2 	 blockNo: 2 	 threadNo: 0 	 threadId: 4
Hello World from GPU's 	 blockDim:2 	 blockNo: 2 	 threadNo: 1 	 threadId: 5
Hello World from GPU's 	 blockDim:2 

---
---
---
# **LP1 - High Performance Computing**
**Compute Unified Device Architecture : CUDA**

CUDA Programming

---
# **Assignment 1**
a)Implement Parallel Reduction using Min, Max, Sum and Average operations.
---

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

#define BLOCK_SIZE 64

#define SOA 512

void randomInit(int* array, int size){
  for (int i = 0; i< size; ++i){
    //array[i] = rand()%size;
    array[i] = i*10;
  }
}

__global__ void kernelReduceMin(int *input, int *results, int n){
  __shared__ int sdata[BLOCK_SIZE]; 
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int tx = threadIdx.x; 

  int temp = 0; 
  if(i< n){
  temp = input[i]; 
  sdata[tx] = temp; 
  }
  __syncthreads();

  for(int stride = blockDim.x>>1; stride > 0; stride >>= 1){
    __syncthreads();
   //printf("i = %d \t tx = %d \t offset = %d \n",i,tx,stride);
    if(tx< stride){
      if(sdata[tx +stride] <sdata[tx]){
        sdata[tx] = sdata[tx + stride];
      }
    }
  }
  
  if(threadIdx.x == 0) 
  { 
    results[blockIdx.x] = sdata[0]; 
  } 
}

__global__ void kernelReduceMax(int *input, int *results, int n){
  __shared__ int sdata[BLOCK_SIZE]; 
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int tx = threadIdx.x; 

  int temp = 0; 
  if(i< n){
  temp = input[i]; 
  sdata[tx] = temp; 
  }
  __syncthreads();

  for(int stride = blockDim.x>>1; stride > 0; stride >>= 1){
    __syncthreads();
   //printf("i = %d \t tx = %d \t offset = %d \n",i,tx,stride);
    if(tx< stride){
      if(sdata[tx +stride] >sdata[tx]){
        sdata[tx] = sdata[tx + stride];
      }
    }
  }
  
  if(threadIdx.x == 0) 
  { 
    results[blockIdx.x] = sdata[0]; 
  } 
}

__global__ void kernelReduceSum(int *input, int *results, int n){
  __shared__ int sdata[BLOCK_SIZE]; 
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int tx = threadIdx.x; 

  int temp = 0; 
  if(i< n){
  temp = input[i]; 
  sdata[tx] = temp; 
  }
  __syncthreads();

  for(int stride = blockDim.x>>1; stride > 0; stride >>= 1){
    __syncthreads();
   //printf("i = %d \t tx = %d \t offset = %d \n",i,tx,stride);
    if(tx< stride){
        sdata[tx] += sdata[tx + stride];
    }
  }
  
  if(threadIdx.x == 0) 
  { 
    results[blockIdx.x] = sdata[0]; 
  } 
}

__global__ void kernelReduceAvg(int *input, int *results, int n){
  __shared__ int sdata[BLOCK_SIZE]; 
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int tx = threadIdx.x; 

  int temp = 0; 
  if(i< n){
  temp = input[i]; 
  sdata[tx] = temp; 
  }
  __syncthreads();

  for(int stride = blockDim.x>>1; stride > 0; stride >>= 1){
    __syncthreads();
   //printf("i = %d \t tx = %d \t offset = %d \n",i,tx,stride);
    if(tx< stride){
        sdata[tx] =(sdata[tx] + sdata[tx + stride])/2;
    }

  }
  
  if(threadIdx.x == 0) 
  {  
    results[blockIdx.x] = sdata[0]; 
  } 
}

/*
void ReduceMin(){}
void ReduceMax(){}
void ReduceSum(){}
void ReduceAvg(){}
*/

  int main(){ 
      
  //1- Declare variables

    int num_blocks = SOA / BLOCK_SIZE; //512/64
    int num_threads=BLOCK_SIZE,i;
    
    int mem_size_a = sizeof(int) * SOA;
    int mem_size_b = sizeof(int) * num_blocks;
    int mem_size_c = sizeof(int) ;
  
  //2- Allocate Memory to variables

    int* h_a = (int*)malloc(sizeof(int) * SOA);
    int* h_b = (int*)malloc(sizeof(int) * num_blocks);
    int* h_c = (int*)malloc(sizeof(int));

    int* d_a;
    cudaMalloc((void**) &d_a, sizeof(int) * SOA);
    int* d_b;
    cudaMalloc((void**) &d_b, sizeof(int) * num_blocks);
    int* d_c;
    cudaMalloc((void**) &d_c, sizeof(int));

  //3-Fill host variables with data

    randomInit(h_a,SOA);

//MIN
    printf("REDUCE MIN:>");

  //4-Copy Data Host to device

    cudaMemcpy(d_a, h_a, sizeof(int) * SOA, cudaMemcpyHostToDevice);
  
  //5-Call Kernel function

    //reduce per-block partial mins
    kernelReduceMin<<<num_blocks, num_threads>>>(d_a,d_b,SOA);
    //then reduce partial mins to a final mix
    kernelReduceMin<<<1, num_blocks>>>(d_b,d_c,num_blocks);
    
  //6-Copy Data Device to Host

    cudaMemcpy(h_b, d_b, mem_size_b, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_c, d_c, mem_size_c, cudaMemcpyDeviceToHost);

  //7-Check results

    printf("\nh_a[]\n");
    for(i=0;i<SOA;i++){
      printf("%d\t",h_a[i]);
    }
    
    printf("\nh_b[] \n");
    for(i=0;i<num_blocks;i++){
      printf("%d\t",h_b[i]);
    }printf("\n");
    
    printf("parallel min=%d\n",*h_c);

//MAX
    printf("REDUCE MAX:>");
    cudaMemcpy(d_a, h_a, sizeof(int) * SOA, cudaMemcpyHostToDevice);

    kernelReduceMax<<<num_blocks, num_threads>>>(d_a,d_b,SOA);
    kernelReduceMax<<<1, num_blocks>>>(d_b,d_c,num_blocks);
    
    cudaMemcpy(h_b, d_b, mem_size_b, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_c, d_c, mem_size_c, cudaMemcpyDeviceToHost);
    
    printf("\nh_a[]\n");
    for(i=0;i<SOA;i++){
      printf("%d\t",h_a[i]);
    }
    
    printf("\nh_b[]\n");
    for(i=0;i<num_blocks;i++){
      printf("%d\t",h_b[i]);
    }printf("\n");
    
    printf("parallel max=%d\n",*h_c);

//SUM
    printf("REDUCE SUM:>");
    cudaMemcpy(d_a, h_a, sizeof(int) * SOA, cudaMemcpyHostToDevice);

    kernelReduceSum<<<num_blocks, num_threads>>>(d_a,d_b,SOA);
    kernelReduceSum<<<1, num_blocks>>>(d_b,d_c,num_blocks);
    
    cudaMemcpy(h_b, d_b, mem_size_b, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_c, d_c, mem_size_c, cudaMemcpyDeviceToHost);
    
    printf("\nh_a[]\n");
    for(i=0;i<SOA;i++){
      printf("%d\t",h_a[i]);
    }
    
    printf("\nh_b[] \n");
    for(i=0;i<num_blocks;i++){
      printf("%d\t",h_b[i]);
    }printf("\n");
    
    printf("parallel Sum=%d\n",*h_c);

//AVG
    printf("REDUCE AVG:>");
    cudaMemcpy(d_a, h_a, sizeof(int) * SOA, cudaMemcpyHostToDevice);

    //reduce per-block partial avg
    kernelReduceAvg<<<num_blocks, num_threads>>>(d_a,d_b,SOA);
    kernelReduceAvg<<<1, num_blocks>>>(d_b,d_c,num_blocks);
    
    cudaMemcpy(h_b, d_b, mem_size_b, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_c, d_c, mem_size_c, cudaMemcpyDeviceToHost);
    
    printf("\nh_a[]\n");
    for(i=0;i<SOA;i++){
      printf("%d\t",h_a[i]);
    }
    
    printf("\nh_b[] \n");
    for(i=0;i<num_blocks;i++){
      printf("%d\t",h_b[i]);
    }printf("\n");
    
    printf("parallel Avg=%d\n",*h_c);

  //8-Deallocate Memory

    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    cudaThreadExit();

  }

REDUCE MIN:>
h_a[]
0	10	20	30	40	50	60	70	80	90	100	110	120	130	140	150	160	170	180	190	200	210	220	230	240	250	260	270	280	290	300	310	320	330	340	350	360	370	380	390	400	410	420	430	440	450	460	470	480	490	500	510	520	530	540	550	560	570	580	590	600	610	620	630	640	650	660	670	680	690	700	710	720	730	740	750	760	770	780	790	800	810	820	830	840	850	860	870	880	890	900	910	920	930	940	950	960	970	980	990	1000	1010	1020	1030	1040	1050	1060	1070	1080	1090	1100	1110	1120	1130	1140	1150	1160	1170	1180	1190	1200	1210	1220	1230	1240	1250	1260	1270	1280	1290	1300	1310	1320	1330	1340	1350	1360	1370	1380	1390	1400	1410	1420	1430	1440	1450	1460	1470	1480	1490	1500	1510	1520	1530	1540	1550	1560	1570	1580	1590	1600	1610	1620	1630	1640	1650	1660	1670	1680	1690	1700	1710	1720	1730	1740	1750	1760	1770	1780	1790	1800	1810	1820	1830	1840	1850	1860	1870	1880	1890	1900	1910	1920	1930	1940	1950	1960	1970	1980	1990	2000	2010	2020	2030	2040	2050	2060	2070	2080	2090	2100	2110	2120	2130	2140	2150	2160	2170	21

b)Write CUDA programs that, given an N-element vector, find 
1. The maximum element in the vector.
2. The minimum element in the vector.
3. The arithmetic mean of the vector.
4. The standard deviation of the values in the vector.

Test for input N and generate a randomized vector V of length N (N should be large). The program should generate output as the two computed maximum values as well as the time taken to find each value.

---

In [ ]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;

__global__ void maximum(int *a,int *b,int n){
	int block=256*blockIdx.x; 
	int max=0;

	for(int i=block;i<min(256+block,n);i++){
		if(max<a[i]){
			max=a[i];	
		}
	}
	b[blockIdx.x]=max;
}

__global__ void minimum(int *a,int *b,int n){
	int block=256*blockIdx.x;
	int mini=7888888;

	for(int i=block;i<min(256+block,n);i++){
		if(mini>a[i]){
			mini=a[i];
		}
	}
	b[blockIdx.x]=mini;
}

__global__ void sum(int *a,int *b,int n){
	int block=256*blockIdx.x;
	int sum=0;

	for(int i=block;i<min(block+256,n);i++){
		sum=sum+a[i];
	}
	b[blockIdx.x]=sum;
}

__global__ void var(int *a,int *b,int n,float mean){
	int block=256*blockIdx.x;
	float sum=0;

	for(int i=block;i<min(block+256,n);i++){
	  sum=sum+(a[i]-mean)*(a[i]-mean);
	}
	b[blockIdx.x]=sum;
}

void maxInVector(){
	int n = 512;
	int a[n];
	cudaEvent_t start,end;

	for(int i=0;i<n;i++){
		a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1){
		maximum<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time=0;
	cudaEventElapsedTime(&time,start,end);	
	int ans[2];
	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The maximum element is "<<ans[0]<<endl;	
	cout<<"The time required do it is ";
	cout<<time<<endl;

}

void minInVector(){
  int n = 512;
	int a[n];

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);

	for(int i=0;i<n;i++){
		a[i]=i+1;
	}
	
	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

  int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);
	cudaEventRecord(start);
	while(n>1){
		minimum<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);
	float time=0;
	cudaEventElapsedTime(&time,start,end);

	int ans[2];
	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The minimum element is "<<ans[0]<<endl;
	
	cout<<"The time required dor it is ";
	cout<<time<<endl;
}

void meanInVector(){
  int n = 512;
	int a[n];
	for(int i=0;i<n;i++){
		a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);

	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);
	int p=n;

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1){
		sum<<<grid,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

  float time=0;
	cudaEventElapsedTime(&time,start,end);
  cout<<"The time required is "<<time<<endl;

	int add[2];
	n=p;

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);
	cout<<"The sum is  "<<add[0]<<endl;

	float mean=0.0f;
	mean=add[0]/(n*1.0f);
	cout<<"The mean is   "<<mean<<endl;
}

void sdInVector(){
  int n = 512;
	int a[n];

	for(int i=0;i<n;i++){
	  a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);

	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);

	int p=n;

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1){
		sum<<<grid,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time=0;
	cudaEventElapsedTime(&time,start,end);
	cout<<"The time required is "<<time<<endl;

	int add[2];
	n=p;

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);
	cout<<"The sum is  "<<add[0]<<endl;

	float mean=0.0f;
	mean=add[0]/(n*1.0f);

	cout<<"The mean is   "<<mean<<endl;
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);
	cudaMalloc(&bd,grids*sizeof(int));

	var<<<grid,block>>>(ad,bd,n,mean);
	n=ceil(n*1.0f/256.0f);	

	sum<<<grid,block>>>(bd,ad,n);

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);

	float sd=sqrt(add[0]/p*1.0f);
	cout<<"The standard deviation is "<<sd<<endl;
}

int main(){
    cout<<"The maximum element in the vector."<<endl;
    maxInVector();
    cout<<endl<<endl<<endl<<"The minimum element in the vector."<<endl;
    minInVector();
    cout<<endl<<endl<<endl<<"The arithmetic mean of the vector."<<endl;
    meanInVector();
    cout<<endl<<endl<<endl<<"The standard deviation of the values in the vector."<<endl;    
    sdInVector();
} 


The maximum element in the vector.
The maximum element is 512
The time required do it is 0.037184



The minimum element in the vector.
The minimum element is 1
The time required dor it is 0.026656



The arithmetic mean of the vector.
The time required is 0.032352
The sum is  131328
The mean is   256.5



The standard deviation of the values in the vector.
The time required is 0.025152
The sum is  131328
The mean is   256.5
The standard deviation is 147.801



# Average

In [ ]:
%%cu
#include<stdio.h>
#include<time.h>

#define SIZE 100

__global__ void sum(const int* __restrict__ input, const int size, int* sumOut)
{
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    atomicAdd(sumOut, input[i]);
    __syncthreads();
}

int main()
{
  int i;
  int a[SIZE];
  int c = 0;
  int *dev_a, *dev_c;
  float gpu_elapsed_time,avg;
  cudaEvent_t gpu_start,gpu_stop;
    
  cudaMalloc((void **) &dev_a, SIZE*sizeof(int));
  cudaMalloc((void **) &dev_c, sizeof(int));
  srand(time(0));
  for( i = 0 ; i < SIZE ; i++)
  {
    a[i] = (rand() % (1000 - 100 + 1)) + 100;
  }
  for( i = 0 ; i < SIZE ; i++)
  {
    printf("%d ",a[i]);
    if (i%10==0 && i!=0){
      printf("\n");
    }
  }
  cudaMemcpy(dev_c , &c, sizeof(int),cudaMemcpyHostToDevice);
  cudaMemcpy(dev_a , a, SIZE*sizeof(int),cudaMemcpyHostToDevice);
  cudaEventCreate(&gpu_start);
  cudaEventCreate(&gpu_stop);
  cudaEventRecord(gpu_start,0);
  sum<<<2,SIZE/2>>>(dev_a,SIZE,dev_c);
  cudaDeviceSynchronize();
  cudaMemcpy(&c, dev_c, sizeof(int),cudaMemcpyDeviceToHost);
  c = c / SIZE;
  cudaEventRecord(gpu_stop, 0);
  cudaEventSynchronize(gpu_stop);
  cudaEventElapsedTime(&gpu_elapsed_time, gpu_start, gpu_stop);
  cudaEventDestroy(gpu_start);
  cudaEventDestroy(gpu_stop);
  printf("avg =  %d ",c);
  printf("\nThe gpu took: %f milli-seconds.\n",gpu_elapsed_time);
    
  printf("\n");
  printf("avg =  %d ",c);
  cudaFree(dev_a);
  cudaFree(dev_c);
  return 0;
}


876 236 113 995 244 500 371 600 343 364 884 
944 502 733 590 728 589 783 786 399 357 
572 208 202 767 393 268 969 677 788 142 
552 320 452 843 465 249 511 361 492 775 
541 435 574 273 321 598 158 103 384 458 
657 252 566 759 919 256 927 887 229 110 
325 978 331 677 821 993 826 331 650 614 
402 191 345 876 661 566 474 720 866 154 
474 522 306 940 577 521 492 800 407 621 
811 632 599 438 605 716 430 728 343 avg =  541 
The gpu took: 0.047744 milli-seconds.

avg =  541 


# Max

In [ ]:
%%cu
#include <stdio.h>
#include <time.h>
#define SIZE 100

__global__ void max(const int* __restrict__ input, const int size, int* maxOut)
{
    int localMax = 0;

    for (int i = threadIdx.x; i < size; i += blockDim.x)
    {
        int val = input[i];
        if (localMax < abs(val))
        {
            localMax = abs(val);
        }
    }
    atomicMax(maxOut, localMax);
    __syncthreads();
}

int main()
{
  int i;
  int a[SIZE];
  int c;
  int *dev_a, *dev_c;
  cudaMalloc((void **) &dev_a, SIZE*sizeof(int));
  cudaMalloc((void **) &dev_c, sizeof(int));
  srand(time(0));
  for( i = 0 ; i < SIZE ; i++)
  {
    a[i] = rand()% 1000;
  }
  for( i = 0 ; i < SIZE ; i++)
  {
    printf("%d ",a[i]);
    if (i%10==0 && i!=0){
      printf("\n");
    }
  }
  c = a[0];
  cudaMemcpy(dev_c , &c, sizeof(int),cudaMemcpyHostToDevice);
  cudaMemcpy(dev_a , a, SIZE*sizeof(int),cudaMemcpyHostToDevice);
  max<<<2,SIZE/2>>>(dev_a,SIZE,dev_c);
  cudaDeviceSynchronize();
  cudaMemcpy(&c, dev_c, sizeof(int),cudaMemcpyDeviceToHost);
  printf("\n");
  printf("max =  %d ",c);
  cudaFree(dev_a);
  cudaFree(dev_c);
  return 0;
}


432 159 104 450 952 409 798 702 467 999 670 
635 243 791 93 478 967 757 752 267 190 
959 189 664 129 826 84 415 633 618 197 
65 778 653 868 730 63 18 784 530 369 
807 517 613 598 610 443 917 368 548 184 
910 507 725 574 636 551 10 404 536 628 
953 602 406 606 822 489 669 840 625 551 
209 432 68 174 30 31 970 299 751 518 
483 13 377 209 587 365 112 949 121 649 
577 74 603 336 681 425 177 702 617 
max =  999 


# Min

In [ ]:
%%cu
#include <stdio.h>
#include <time.h>
#define SIZE 100

__global__ void min(const int* __restrict__ input, const int size, int* minOut)
{
    int localMin = 1000;

    for (int i = threadIdx.x; i < size; i += blockDim.x)
    {
        int val = input[i];
        if (localMin > abs(val))
        {
            localMin = abs(val);
        }
    }
    atomicMin(minOut, localMin);
    __syncthreads();
}

int main()
{
  int i;
  int a[SIZE];
  int c;
  int *dev_a, *dev_c;
  cudaMalloc((void **) &dev_a, SIZE*sizeof(int));
  cudaMalloc((void **) &dev_c, sizeof(int));
  srand(time(0));
  for( i = 0 ; i < SIZE ; i++)
  {
    a[i] = (rand() % (1000 - 100 + 1)) + 100;
  }
  for( i = 0 ; i < SIZE ; i++)
  {
    printf("%d ",a[i]);
    if (i%10==0 && i!=0){
      printf("\n");
    }
  }
  c = a[0];
  cudaMemcpy(dev_c , &c, sizeof(int),cudaMemcpyHostToDevice);
  cudaMemcpy(dev_a , a, SIZE*sizeof(int),cudaMemcpyHostToDevice);
  min<<<2,SIZE/2>>>(dev_a,SIZE,dev_c);
  cudaDeviceSynchronize();
  cudaMemcpy(&c, dev_c, sizeof(int),cudaMemcpyDeviceToHost);
  printf("\n");
  printf("min =  %d ",c);
  cudaFree(dev_a);
  cudaFree(dev_c);
  return 0;
}


615 214 374 155 823 863 752 457 401 863 344 
443 553 351 169 875 723 802 559 400 840 
925 570 443 115 471 360 243 748 191 146 
362 306 718 418 128 580 466 782 881 328 
423 620 178 674 986 953 693 183 511 994 
319 732 860 663 747 331 319 286 375 411 
630 638 617 347 956 942 827 421 723 103 
946 145 920 420 115 905 372 709 384 179 
702 604 207 561 563 250 188 782 437 464 
192 967 101 105 313 353 947 436 971 
min =  101 


# Sum

In [ ]:
%%cu
#include<stdio.h>

__global__ void sum(int* input, int* sumOut) {
        int i = threadIdx.x + blockIdx.x * blockDim.x;
        for(int j = 0; j < 100/(blockDim.x*gridDim.x); j++){
                if (i < 100){
                        atomicAdd(sumOut, input[i+(j*blockDim.x*gridDim.x)]);
                      //  printf("NUM:%d Thread: %d ||\n",input[i+(j*blockDim.x*gridDim.x)],i);
                }
        }
        __syncthreads();
}

int main() {
        int n = 100;
        int input[n];
        int sumO = 0;
        int i = 0;
        for(i = 0; i < n; i++){
                input[i] = (rand() % 1000) + 100;
                /*if(i % 10 == 0){
                        printf("\n");
                }
                printf("%d ",input[i]);*/
        }
        
        int* d_input;
        int* d_sum;
        
        cudaMalloc((void**)&d_input, n * sizeof(int));
        cudaMalloc((void**)&d_sum, sizeof(int));
        
        cudaMemcpy(d_input, &input, n * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_sum, &sumO, sizeof(int), cudaMemcpyHostToDevice);
        
        sum<<<2,10>>>(d_input, d_sum);
        
        cudaMemcpy(&sumO, d_sum, sizeof(int), cudaMemcpyDeviceToHost);
        
        printf("\nSum: %d",sumO);
        
        cudaFree(d_sum);
        cudaFree(d_input);
        
        return 0;
}



Sum: 57684


# **Assignment 2**

Vector and Matrix Operations :

Design parallel algorithm to
1. Add two large vectors
2. Multiply Vector and Matrix
3. Multiply two N × N arrays using n2 processors.

---

In [ ]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda.h>
#define N 4
#define TPB 2
 
// CUDA kernel. Each thread takes care of one element of c
__global__ void vecAdd(double *a, double *b, double *c, int n){
    // Get our global thread ID
    int id = blockIdx.x*blockDim.x+threadIdx.x; 
    // Make sure we do not go out of bounds
    if (id < n)
        c[id] = a[id] + b[id];
}

__global__ void matrixMultiplication(int *a, int *b, int *c, int n){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    int i;
    if (row < n && col < n){
      for( i=0; i<n ;i++){
        sum += a[row *n +i] *  b[i * n+ col];
      }
      c[row *N +col]=sum;
    }
}
 
void vecAdd(){
    // Size of vectors
    int n = 100000;
 
    // Host input vectors
    double *h_a;
    double *h_b;
    //Host output vector
    double *h_c;
 
    // Device input vectors
    double *d_a;
    double *d_b;
    //Device output vector
    double *d_c;
 
    // Size, in bytes, of each vector
    size_t bytes = n*sizeof(double);
 
    // Allocate memory for each vector on host
    h_a = (double*)malloc(bytes);
    h_b = (double*)malloc(bytes);
    h_c = (double*)malloc(bytes);
 
    // Allocate memory for each vector on GPU
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
 
    int i;
    // Initialize vectors on host
    for( i = 0; i < n; i++ ){
        h_a[i] = i;
        h_b[i] = i;
    }
 
    // Copy host vectors to device
    cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);
 
    int blockSize, gridSize;
 
    // Number of threads in each thread block
    blockSize = 1024;
 
    // Number of thread blocks in grid
    gridSize = (int)ceil((float)n/blockSize);
 
    // Execute the kernel
    vecAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);
 
    // Copy array back to host
    cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );
 
    // Sum up vector c and print result divided by n, this should equal 1 within error
    double sum = 0;
    for(i=0; i<n; i++)
        sum += h_c[i];
    printf("Sum: %f\n", sum/n);
 
    // Release device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
 
    // Release host memory
    free(h_a);
    free(h_b);
    free(h_c);
}

void matMul(){
  int *h_a, *h_b, *h_c;
  int *d_a, *d_b, *d_c;

  int size=sizeof(int)*N*N;
  cudaEvent_t start,end;
  float time=0;
  h_a=(int*)malloc(size);
  h_b=(int*)malloc(size);
  h_c=(int*)malloc(size);
  cudaEventCreate(&start);
  cudaEventCreate(&end);
  cudaEventRecord(start);

  cudaMalloc(&d_a, size);
  cudaMalloc(&d_b, size);
  cudaMalloc(&d_c, size);

  for (int i = 0; i < N*N; i++){
 	  h_a[i] = random() % N;
 	  h_b[i] = random() % N;
  }

  printf("\nMatrix A =>\n\n");
  for (int i = 0; i < N; i++){
    for(int j = 0;j<N; j++){
      printf("%d ",h_a[i*N + j]);
    }
 	  printf("\n");
   }

  printf("\nMatrix B =>\n\n");
   for (int i = 0; i < N; i++){
      for(int j = 0;j<N; j++){
        printf("%d ",h_b[i*N + j]);
      }
      printf("\n");
    }



  cudaMemcpy( d_a, h_a, size, cudaMemcpyHostToDevice);
  cudaMemcpy( d_b, h_b, size, cudaMemcpyHostToDevice);

  int BLOCK_SIZE=N / TPB;

  dim3 GridSize(BLOCK_SIZE, BLOCK_SIZE);
  dim3 BlockSize(TPB, TPB);

  matrixMultiplication<<<GridSize,BlockSize>>>(d_a, d_b, d_c, N);

  cudaMemcpy( h_c, d_c, size, cudaMemcpyDeviceToHost );
  cudaEventRecord(end);
  cudaEventSynchronize(end);
  cudaEventElapsedTime(&time,start,end);
  printf("\nMatrix C =>\n\n");

  for (int i = 0; i < N; i++){
    for(int j = 0;j<N; j++){
      printf("%d ",h_c[i*N + j]);
    }
    printf("\n");
  }

  printf("Time taken to perform %d by %d matrix mul is: %lf ms",N,N,time);

   cudaFree(d_a);
   cudaFree(d_b);
   cudaFree(d_c);

   free(h_a);
   free(h_b);
   free(h_c);
}


int main( int argc, char* argv[] )
{
    printf("Add two large vectors \n");
    vecAdd();    
    //printf("\n\n\n Multiply Vector and Matrix \n");    
    printf("\n\n\n Multiply two N × N arrays using n2 processors\n");
    matMul();
 return 0;
}

Add two large vectors 
Sum: 99999.000000



 Multiply two N × N arrays using n2 processors

Matrix A =>

3 1 1 2 
1 2 2 3 
0 0 3 3 
2 2 3 1 

Matrix B =>

2 3 3 0 
1 3 3 2 
2 0 0 1 
2 3 3 2 

Matrix C =>

13 18 18 7 
14 18 18 12 
12 9 9 9 
14 15 15 9 
Time taken to perform 4 by 4 matrix mul is: 0.151904 ms


#**OpenMP**

# **Assignment 3** : OpenMp
Parallel Sorting Algorithms for Bubble Sort and Merger Sort, based on existing sequential algorithms, design and implement parallel algorithm utilizing all resources available.
---

In [ ]:
%%cu
#include <iostream>
#include <omp.h>
using namespace std;

void bubblesort(int a[],int n){
  for (int i = 0; i < n; i++) {
    int first = i%2;
    #pragma omp parallel for shared(a,first)
    for (int j = first; j < n-1; j+=2) {
        if (a[j] > a[j+1]) {
          int temp = a[j+1];
          a[j+1] = a[j];
          a[j] = temp;
      }
    }
  }
  cout<<"Sorted list:\n";
  for (int i = 0; i < n; i++) {
    cout<<a[i]<<"\n";
  }
}

int main(){
  int n;
  clock_t t1,t2;
  cout<<"Enter total number of elements:\n";
  //cin>>n;
  n=100;
  int a[n];
  cout<<"Storing elements in descending order....\n\n";
  for(int i = 0 ; i < n; i++){
    a[i] = n-i;
  }
  cout<<"Actual list:\n";
  for (int i = 0; i < n; i++) {
    cout<<a[i]<<"\n";
  }
  cout<<"\n";
  t1 = clock();
  bubblesort(a,n);
  t2 = clock();
  //cout<<t1;
  
  printf("\nExecution Time:%f",(double)(t2-t1)/CLOCKS_PER_SEC);
}


Enter total number of elements:
Storing elements in descending order....

Actual list:
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1

Sorted list:
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100

Execution Time:0.000050


In [ ]:
%%cu
#include<iostream>
#include<omp.h>
using namespace std;

void merge(int arr[], int l, int m, int r)
{
	int i, j, k;
	int n1 = m - l + 1;
	int n2 = r - m;

	int L[n1], R[n2];

	for (i = 0; i < n1; i++)
		L[i] = arr[l + i];
	for (j = 0; j < n2; j++)
		R[j] = arr[m + 1 + j];

	i = 0;
	j = 0;
	k = l;
	while (i < n1 && j < n2)
	{
		if (L[i] <= R[j])
		{
			arr[k] = L[i];
			i++;
		}
		else
		{
			arr[k] = R[j];
			j++;
		}
		k++;
	}


	while (i < n1)
	{
		arr[k] = L[i];
		i++;
		k++;
	}


	while (j < n2)
	{
		arr[k] = R[j];
		j++;
		k++;
	}
}


void mergeSort(int arr[], int l, int r)
{
	if (l < r)
	{
		int m = l+(r-l)/2;
    #pragma omp parallel sections
    {
      #pragma omp section
      {
        mergeSort(arr, l, m);
      }
      #pragma omp section
      {
        mergeSort(arr, m+1, r);
      }
    }
		merge(arr, l, m, r);
	}
}

void printArray(int A[], int size)
{
	int i;
	for (i=0; i < size; i++)
		cout<<A[i]<<" ";
  cout<<"\n";
}

int main()
{
  int n;
  clock_t t1,t2;
  cout<<"Enter total number of elements:\n";
  cin>>n;
  n=199;
  int a[n];
  cout<<"Storing elements in descending order....\n\n";
  for(int i = 0 ; i < n; i++){
    a[i] = n-i;
  }

  cout<<"Actual list:\n";
	printArray(a, n);
  t1 = clock();
	mergeSort(a, 0, n - 1);
  t2 = clock();
  cout<<"Sorted list:\n";
	printArray(a, n);

  printf("\nExecution Time:%f",(double)(t2-t1)/CLOCKS_PER_SEC);
  return 0;
}


Enter total number of elements:
Storing elements in descending order....

Actual list:
199 198 197 196 195 194 193 192 191 190 189 188 187 186 185 184 183 182 181 180 179 178 177 176 175 174 173 172 171 170 169 168 167 166 165 164 163 162 161 160 159 158 157 156 155 154 153 152 151 150 149 148 147 146 145 144 143 142 141 140 139 138 137 136 135 134 133 132 131 130 129 128 127 126 125 124 123 122 121 120 119 118 117 116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 
Sorted list:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 7

# **Assignment 4** : OpenMp

Parallel Search Algorithm-

Design and implement parallel algorithm utilizing all resources available. For
1. Binary Search for Sorted Array
2. Depth-First Search (tree or an undirected graph ) : by us
3. Breadth-First Search ( tree or an undirected graph)

In [ ]:
%%cu
#include<iostream>
#include<omp.h>
#include<time.h>
using namespace std;

int binary_search(int a[],int low,int high,int key){
  int loc = -1;
  int mid;
  while(low<=high){
    mid = (high + low )/2;
    if (a[mid] == key) {
      loc = mid;
      break;
    }
    else {
      #pragma omp parallel sections
      {
        #pragma omp section
        {
          if(a[mid]<key){
            low = mid+1;
          }
        }
        #pragma omp section
        {
          if(a[mid]>key){
            high = mid-1;
          }
        }
      }
    }
  }
  return loc;
}

int main(){

  int a[1000000];
  clock_t t1,t2;
  int key = 0;
  int loc,i;
  for (int i = 0; i < 1000000; i++) {
    a[i] = i;
  }
  cout<<"Enter key to search: \n";
  //cin>>key;
  key = 56798;
  t1 = clock();
  loc = binary_search(a,0,1000000,key);
  t2 = clock();
  if (loc == -1) {
    cout<<"Key not found\n";
  } else {
    cout<<"Key found at "<<loc<<"\n";
   // cout<<"Running thread "<<omp_get_thread_num()<<"\n";
  }
  //cout<<"Execution time: "<<t1<<"\t"<<t2<<"\t"<<t2-t1<<"\n";
  printf("\nExecution Time:%f",(double)(t2-t1)/CLOCKS_PER_SEC);

  return 0 ;
}

Enter key to search: 
Key found at 56798

Execution Time:0.000002
